In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BankingDataAssessment").getOrCreate()

25/04/16 15:24:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [84]:
customer_data = [
    (1, "John Doe", "New York", "john.doe@example.com"),
    (2, "Alice Smith", "Los Angeles", "alice.smith@example.com"),
    (3, "Bob Johnson", "Chicago", "bob.johnson@example.com"),
    (4, "Charlie Brown", "Houston", "charlie.brown@example.com")
]
colum = ["CustomerId","Name","City","Email"]
cust_df = spark.createDataFrame(customer_data,colum)

In [85]:
cust_df.show()

+----------+-------------+-----------+--------------------+
|CustomerId|         Name|       City|               Email|
+----------+-------------+-----------+--------------------+
|         1|     John Doe|   New York|john.doe@example.com|
|         2|  Alice Smith|Los Angeles|alice.smith@examp...|
|         3|  Bob Johnson|    Chicago|bob.johnson@examp...|
|         4|Charlie Brown|    Houston|charlie.brown@exa...|
+----------+-------------+-----------+--------------------+



In [86]:
cust_df.limit(3)

DataFrame[CustomerId: bigint, Name: string, City: string, Email: string]

In [87]:
cust_df.show(2)

+----------+-----------+-----------+--------------------+
|CustomerId|       Name|       City|               Email|
+----------+-----------+-----------+--------------------+
|         1|   John Doe|   New York|john.doe@example.com|
|         2|Alice Smith|Los Angeles|alice.smith@examp...|
+----------+-----------+-----------+--------------------+
only showing top 2 rows



In [88]:
transaction_data = [
    (1, 1001, "2025-01-01", 2500.0),
    (2, 1002, "2025-01-02", 5000.0),
    (1, 1003, "2025-01-03", 1500.0),
    (3, 1004, "2025-02-01", 1200.0),
    (2, 1005, "2025-02-05", 3000.0),
    (4, 1006, "2025-02-10", 4500.0)
]
 
transaction_columns = ["CustomerID", "TransactionID", "TransactionDate", "Amount"]

In [89]:
tran_df = spark.createDataFrame(transaction_data, transaction_columns)

In [90]:
tran_df.show()

+----------+-------------+---------------+------+
|CustomerID|TransactionID|TransactionDate|Amount|
+----------+-------------+---------------+------+
|         1|         1001|     2025-01-01|2500.0|
|         2|         1002|     2025-01-02|5000.0|
|         1|         1003|     2025-01-03|1500.0|
|         3|         1004|     2025-02-01|1200.0|
|         2|         1005|     2025-02-05|3000.0|
|         4|         1006|     2025-02-10|4500.0|
+----------+-------------+---------------+------+



In [91]:
'''from pyspark.sql.types import *
from pyspark.sql import functions as F
x = tran_df.groupBy("CustomerID").agg(F.count("*"))
tran_df = tran_df.withColumn("TransactionDate",F.to_date(F.col("TransactionDate"),"yyyy-MM-dd"))

SyntaxError: incomplete input (3127207377.py, line 1)

In [92]:
x.show()

+----------+-------------+-----------+
|CustomerID|         Name|Total_spent|
+----------+-------------+-----------+
|         1|     John Doe|     4000.0|
|         2|  Alice Smith|     8000.0|
|         3|  Bob Johnson|     1200.0|
|         4|Charlie Brown|     4500.0|
+----------+-------------+-----------+



In [93]:
tran_df.printSchema()

root
 |-- CustomerID: long (nullable = true)
 |-- TransactionID: long (nullable = true)
 |-- TransactionDate: string (nullable = true)
 |-- Amount: double (nullable = true)



In [94]:
joined = cust_df.join(tran_df,on="CustomerID",how="inner")

In [95]:
joined.show()

+----------+-------------+-----------+--------------------+-------------+---------------+------+
|CustomerId|         Name|       City|               Email|TransactionID|TransactionDate|Amount|
+----------+-------------+-----------+--------------------+-------------+---------------+------+
|         1|     John Doe|   New York|john.doe@example.com|         1001|     2025-01-01|2500.0|
|         1|     John Doe|   New York|john.doe@example.com|         1003|     2025-01-03|1500.0|
|         2|  Alice Smith|Los Angeles|alice.smith@examp...|         1002|     2025-01-02|5000.0|
|         2|  Alice Smith|Los Angeles|alice.smith@examp...|         1005|     2025-02-05|3000.0|
|         3|  Bob Johnson|    Chicago|bob.johnson@examp...|         1004|     2025-02-01|1200.0|
|         4|Charlie Brown|    Houston|charlie.brown@exa...|         1006|     2025-02-10|4500.0|
+----------+-------------+-----------+--------------------+-------------+---------------+------+



In [96]:
print("Joined Data (Customers + Transactions):")

Joined Data (Customers + Transactions):


In [97]:
joined.filter(F.col("Amount")>3000).show()


+----------+-------------+-----------+--------------------+-------------+---------------+------+
|CustomerId|         Name|       City|               Email|TransactionID|TransactionDate|Amount|
+----------+-------------+-----------+--------------------+-------------+---------------+------+
|         2|  Alice Smith|Los Angeles|alice.smith@examp...|         1002|     2025-01-02|5000.0|
|         4|Charlie Brown|    Houston|charlie.brown@exa...|         1006|     2025-02-10|4500.0|
+----------+-------------+-----------+--------------------+-------------+---------------+------+



In [124]:
x = joined.groupBy("CustomerID").agg(F.sum(F.col("Amount")).alias("Total_amount_spent"))
final_df = joined.join(x,on="CustomerID")
final_df.show()

+----------+-------------+-----------+--------------------+-------------+---------------+------+------------------+
|CustomerId|         Name|       City|               Email|TransactionID|TransactionDate|Amount|Total_amount_spent|
+----------+-------------+-----------+--------------------+-------------+---------------+------+------------------+
|         1|     John Doe|   New York|john.doe@example.com|         1003|     2025-01-03|1500.0|            4000.0|
|         1|     John Doe|   New York|john.doe@example.com|         1001|     2025-01-01|2500.0|            4000.0|
|         2|  Alice Smith|Los Angeles|alice.smith@examp...|         1005|     2025-02-05|3000.0|            8000.0|
|         2|  Alice Smith|Los Angeles|alice.smith@examp...|         1002|     2025-01-02|5000.0|            8000.0|
|         3|  Bob Johnson|    Chicago|bob.johnson@examp...|         1004|     2025-02-01|1200.0|            1200.0|
|         4|Charlie Brown|    Houston|charlie.brown@exa...|         1006

In [117]:
#x = final_df.groupBy("CustomerID","Name").agg(F.sum(F.col("Amount")).alias("Total_spent"))
#x.show()

+----------+-------------+-----------+
|CustomerID|         Name|Total_spent|
+----------+-------------+-----------+
|         1|     John Doe|     4000.0|
|         2|  Alice Smith|     8000.0|
|         3|  Bob Johnson|     1200.0|
|         4|Charlie Brown|     4500.0|
+----------+-------------+-----------+



In [125]:
y = final_df.groupBy("CustomerID","Name").agg(F.count(F.col("TransactionID")).alias("Number_of_transactions"))
final_dff = joined.join(y,on="CustomerID",how="inner")
final_dff.show()

[Stage 361:============>    (3 + 1) / 4][Stage 362:>                (0 + 3) / 4]

+----------+-------------+-----------+--------------------+-------------+---------------+------+-------------+----------------------+
|CustomerId|         Name|       City|               Email|TransactionID|TransactionDate|Amount|         Name|Number_of_transactions|
+----------+-------------+-----------+--------------------+-------------+---------------+------+-------------+----------------------+
|         1|     John Doe|   New York|john.doe@example.com|         1003|     2025-01-03|1500.0|     John Doe|                     2|
|         1|     John Doe|   New York|john.doe@example.com|         1001|     2025-01-01|2500.0|     John Doe|                     2|
|         2|  Alice Smith|Los Angeles|alice.smith@examp...|         1005|     2025-02-05|3000.0|  Alice Smith|                     2|
|         2|  Alice Smith|Los Angeles|alice.smith@examp...|         1002|     2025-01-02|5000.0|  Alice Smith|                     2|
|         3|  Bob Johnson|    Chicago|bob.johnson@examp...|   

In [120]:
final_dff.filter((F.col("TransactionDate") >= "2025-01-01") & (F.col("TransactionDate") <= "2025-01-31")).show()

[Stage 311:====>            (1 + 3) / 4][Stage 312:>                (0 + 1) / 4]

+----------+-----------+-----------+--------------------+-------------+---------------+------+-----------+----------------------+
|CustomerId|       Name|       City|               Email|TransactionID|TransactionDate|Amount|       Name|Number_of_transactions|
+----------+-----------+-----------+--------------------+-------------+---------------+------+-----------+----------------------+
|         1|   John Doe|   New York|john.doe@example.com|         1003|     2025-01-03|1500.0|   John Doe|                     2|
|         1|   John Doe|   New York|john.doe@example.com|         1001|     2025-01-01|2500.0|   John Doe|                     2|
|         2|Alice Smith|Los Angeles|alice.smith@examp...|         1002|     2025-01-02|5000.0|Alice Smith|                     2|
+----------+-----------+-----------+--------------------+-------------+---------------+------+-----------+----------------------+

